## How to run this
#### Install Julia
https://julialang.org/downloads/platform.html
#### Install Julia Kernel in jupyter
Once Julia is installed, open a console :
```Bash
$julia
$julia> Pkg.add("IJulia")
```
This will install the Julia kernel in your Jupyter environment. You can now create a new notebook with the Julia kernel. Autocompletion should work (can depend on pyreadline)
More information:
https://github.com/JuliaLang/IJulia.jl

#### CPython vs Julia
The speed (in this use case) is the result of Julia's interpreter/compiler being able to deduce the type of arguments and variables from a lexical scan. Explicit annotation is possible (for example Z could suffice with Array{Int8, 1} instead of Array{Int64, 1}, but then we're not doing the same anymore as the CPython example.

In [32]:
### Three versions of the same functionality, in the end the cost of preallocating Y is the bottleneck, always

# Optionally specify type : Name::Type
# ! as a postfix means : this function will not alter it's arguments (pass by pointer)
function loop!(X, H, W, Z)
    Y = similar(X, (H,W))     # Allocates same array as first argument without initialization
    @simd for y in 1:W        # Column major order in Julia, so row index changes faster.
        @simd for x in 1:H    # Enables SIMD instructions if applicable
           @inbounds Y[x,y] = Z[X[x,y]] # Removes bound checks in compiled code for maximum speed, does not make an impact here
        end
    end
    return Y
end

# The same as before, but now pass Y in by argument
function loop3(X, H, W, Z, Y)
    @simd for y in 1:W
        @simd for x in 1:H
           @inbounds Y[x,y] = Z[X[x,y]]
        end
    end
end

# Slicing assignment, turns out this is slower since it creates copies internally, todo: figure out to avoid this
function loop2!(X, H, W, Z)
    return Z[X[1:H, 1:W]]
end;

In [35]:
# Bigger example.
H = 5000
W = 5000
srand(0)
X = rand(1:6, (H, W))
Z = [10,11,12,13,14,15];

In [36]:
Y = loop!(X, H, W, Z); # Record Y for validation.


Note: The value of an expression is the expression itself, so in this case Y is printed, unless we silence this by adding";". This is also true for function, the return value (unless you use return) is always the last expression of a function body.

In [48]:

println("Basic implementation")
for i in 1:10
    @time Y = loop!(X, H, W, Z)
end
@printf "Types in use are X %s Y %s Z %s" typeof(X) typeof(Y) typeof(Z)

Basic implementation
  0.055833 seconds (2 allocations: 190.735 MiB, 2.68% gc time)
  0.046620 seconds (2 allocations: 190.735 MiB, 2.53% gc time)
  0.037611 seconds (2 allocations: 190.735 MiB, 2.75% gc time)
  0.042557 seconds (2 allocations: 190.735 MiB, 2.22% gc time)
  0.038147 seconds (2 allocations: 190.735 MiB, 2.37% gc time)
  0.031778 seconds (2 allocations: 190.735 MiB, 2.64% gc time)
  0.031687 seconds (2 allocations: 190.735 MiB, 2.65% gc time)
  0.038285 seconds (2 allocations: 190.735 MiB, 2.43% gc time)
  0.032675 seconds (2 allocations: 190.735 MiB, 2.56% gc time)
  0.031885 seconds (2 allocations: 190.735 MiB, 2.60% gc time)
Types in use are X Array{Int64,2} Y Array{Int64,2} Z Array{Int64,1}

Note the initial GC time, that goes to zero after 2-3 iterations, it's reusing the allocated memory. If we execute the function once, before looping, it will almost immediately go to zero, since it can show that nothing is using Y, so it can reuse the memory. In addition, that memory is cache-hot (since the indices don't change.

You can see that Julia 'knows' the types to be used here. These are the same as CPython's long, signed 64 bit integers. We could cheat here and use {Int8,2} for Y but that's no longer equivalent to the Python use case, and it's not guaranteed to be faster (depends on if/how LLVM will allocate an int per word etc)

##### Slicing
Test if assignment by slicing is faster. 

In [38]:
Y2 = loop2!(X,H,W,Z);

In [39]:
println("Slice operators")
for i in 1:10
    @time Y2 = loop2!(X,H,W,Z)
end

Slice operators
  0.164668 seconds (7 allocations: 381.470 MiB, 38.84% gc time)
  0.152049 seconds (7 allocations: 381.470 MiB, 38.57% gc time)
  0.106229 seconds (7 allocations: 381.470 MiB, 3.46% gc time)
  0.095065 seconds (7 allocations: 381.470 MiB, 1.71% gc time)
  0.091430 seconds (7 allocations: 381.470 MiB, 1.77% gc time)
  0.103734 seconds (7 allocations: 381.470 MiB, 1.51% gc time)
  0.096250 seconds (7 allocations: 381.470 MiB, 1.77% gc time)
  0.100903 seconds (7 allocations: 381.470 MiB, 1.64% gc time)
  0.095786 seconds (7 allocations: 381.470 MiB, 1.80% gc time)
  0.089626 seconds (7 allocations: 381.470 MiB, 1.72% gc time)


Note the doubling of memory, and the runtime. Clearly the slicing is using temporary arrays.

In [40]:
Y == Y2

true

Let's see what happens if we remove allocation from the loop.

In [41]:
Y3 = similar(Y);

In [42]:
loop3(X,H,W,Z,Y3)
for i in 1:10
    print("Isolating the allocation")
#     Y3 = similar(Y)
    @time loop3(X,H,W,Z,Y3)
end

Isolating the allocation  0.033496 seconds
Isolating the allocation  0.041041 seconds
Isolating the allocation  0.033830 seconds
Isolating the allocation  0.040960 seconds
Isolating the allocation  0.043166 seconds
Isolating the allocation  0.033897 seconds
Isolating the allocation  0.040489 seconds
Isolating the allocation  0.033887 seconds
Isolating the allocation  0.033774 seconds
Isolating the allocation  0.040962 seconds


Note how the GC isn't mentioned anymore in the timing, it's extracted to outside the loop.

In [43]:
Y3 == Y

true